In [1]:
from matplotlib.collections import LineCollection
from matplotlib import pyplot as plt
from matplotlib import cm, colors
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import PCA
from hmmlearn import hmm
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

np.set_printoptions(linewidth=300, suppress=True)
pd.options.mode.chained_assignment = None  # default='warn'

from src.experiments.util_experiments import *
from src.oom import DiscreteValuedOOM, ContinuousValuedOOM
from src.oom.discrete_observable import DiscreteObservable
from src.oom.util.few_step_prediction import quantify_distribution, kl_divergence, fix_pvec
from src.oom.util.random_sparse import _generate_sparse_full_rank_matrix, _generate_observable_compound
from src.oom.util.learning_discrete import estimate_matrices_discrete_fixed
from src.oom.util.learning_continuous import estimate_matrices_continuous
from src.oom.util.numrank import numerical_rank_binomial, numerical_rank_frob_mid_spec

In [2]:
# del sys.modules['src.oom.util.learning_discrete']
# del sys.modules['src']
# 
# del sys.modules['src.oom.util.numrank']
# del numerical_rank_binomial
# del numerical_rank_frob_mid_spec

In [3]:
def dvoom_from_hmm(hmm_to_convert):
    d = hmm_to_convert.n_components # dimension
    n = hmm_to_convert.n_features   # observables
    
    lf = np.asmatrix(np.ones(shape=(1, d)))
    ss = np.asmatrix(hmm_to_convert.get_stationary_distribution()).T

    M = hmm_to_convert.transmat_
    E = hmm_to_convert.emissionprob_
    obs = [str(i+1) for i in range(n)]
    ops = [np.matrix(M).T * np.asmatrix(np.diag(E[:, col])) for col in range(n)]
    
    return DiscreteValuedOOM(
        dim               = d,
        linear_functional = lf,
        start_state       = ss,
        obs_ops           = dict(zip(obs, ops))
    )

## Assess pre-existing HMM creations

In [ ]:
# (1) Create a random discrete-time HMM with 3 hidden states and 3 possible outputs
np.random.seed(42)  # For reproducibility

# Define parameters
n_states = 4
n_observations = 3

# Random transition matrix (rows = current state, columns = next state)
transmat = np.random.dirichlet(np.ones(n_states), size=n_states)
transmat /= transmat.sum(axis=1, keepdims=True)  # Ensure row-stochastic

# Random emission probabilities (rows = states, columns = observations)
emission_probs = np.random.dirichlet(np.ones(n_observations), size=n_states)

# Create the true HMM model
true_model = hmm.CategoricalHMM(n_components=n_states)
true_model.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])  # Start in state 0
true_model.transmat_ = transmat
true_model.emissionprob_ = emission_probs
true_model.startprob_ = true_model.get_stationary_distribution()

true_model.emissionprob_
# # (2) Generate training and test sequences
# train_seq, train_states = true_model.sample(100000)
# test_seq, test_states = true_model.sample(100000)

# # (3) Compute NLL of the test sequence under the true model
# true_nll = -true_model.score(test_seq) / test_seq.shape[0]
# print(f"True Model NLL on Test Sequence: {true_nll:.8f}")

# # (4) Train a new HMM on the training sequence
# learned_model = hmm.CategoricalHMM(n_components=n_states, n_iter=100)
# learned_model.fit(train_seq)

# # (5) Compute NLL of the learned model on the test sequence
# learned_nll = -learned_model.score(test_seq) / test_seq.shape[0]
# print(f"Learned Model NLL on Test Sequence: {learned_nll:.8f}")

In [ ]:
# seed = 2
# n = 2
# d = 2
# density = 0.8

oom_model = make_source(name='S_3', case='discrete')
n = len(oom_model.observables)
d = oom_model.dim
density = 0.4
seed = 44 # for S_3

rng = np.random.default_rng(seed = seed)
rvs = sp.stats.uniform(loc = 0.01, scale = 1).rvs
mu = _generate_sparse_full_rank_matrix(dim = d, sparsity = 1-density, rng = rng, rvs = rvs)
Os = _generate_observable_compound(nrows = n, ncols = d, sparsity = 1-density, rng = rng, rvs = rvs)
hmm_model = hmm.CategoricalHMM(n_components = d)
hmm_model.startprob_ = np.zeros(shape = (d, 1))
hmm_model.startprob_[0] = 1
hmm_model.transmat_ = mu
hmm_model.emissionprob_ = Os.T
hmm_model.startprob_ = np.asarray(oom_model.start_state).flatten()

In [ ]:
print(oom_model)

In [ ]:
# OOM generation
oom_gen_train = oom_model.generate(5000).sequence
oom_nll_oom_gen_train = oom_model.compute(oom_gen_train).nll_list[-1]

oom_gen_train_modforhmm = np.array([ord(obs.uid[1:])-ord('a') for obs in oom_gen_train]).reshape(-1, 1)
hmm_nll_oom_gen_train = -hmm_model.score(oom_gen_train_modforhmm) / (oom_gen_train_modforhmm.shape[0] * np.log(2))

# HMM generation
hmm_gen_train, _ = hmm_model.sample(5000)
hmm_nll_hmm_gen_train = -hmm_model.score(hmm_gen_train) / (hmm_gen_train.shape[0] * np.log(2))

hmm_gen_train_modforoom = [DiscreteObservable( chr(i + ord('a')) ) for i in list(hmm_gen_train.flatten())]
oom_nll_hmm_gen_train = oom_model.compute(hmm_gen_train_modforoom).nll_list[-1]

In [ ]:
oom_nll_oom_gen_train, hmm_nll_oom_gen_train, hmm_nll_hmm_gen_train, oom_nll_hmm_gen_train

In [ ]:
train_seq = oom_model.generate(10 ** 5)
print("Done generating train_seq")
test_seqs = [oom_model.generate(5 * 10 ** 3) for _ in range(5)]
print("Done generating test_seq")

oom_model_nlls = [oom_model.compute(test_seq.sequence).nll_list[-1] for test_seq in tqdm(test_seqs)]
oom_model_mean_nll = np.mean(oom_model_nlls)
oom_model_std_nll = np.std(oom_model_nlls)
print(f"True Model NLL on Test Sequence: {oom_model_mean_nll:.4f}+-{oom_model_std_nll:.4f}")

In [ ]:
fsp5_source = fix_pvec(quantify_distribution(5, oom_model.start_state, oom_model.operators, oom_model.lin_func))
train_seq_modforhmm = np.array([ord(obs.uid[1:])-ord('a') for obs in train_seq.sequence]).reshape(-1, 1)
test_seqs_modforhmm = [
    np.array([ord(obs.uid[1:])-ord('a') for obs in test_seq.sequence]).reshape(-1, 1)
    for test_seq in test_seqs
]

In [ ]:
np.seterr(divide='ignore')

best_hmm, best_hmm_kl, best_hmm_nll = None, np.inf, np.inf
best_oom, best_oom_kl, best_oom_nll = None, np.inf, np.inf

for seqlength_spec in range(4, 10 + 1):#, 10**6, 10**7, 10**8]:
    seqlength = int(10 ** (seqlength_spec/2))
    len_cwords = 2
    len_iwords = 3
    
    estimated_matrices = estimate_matrices_discrete_fixed(
        sequence   = train_seq.sequence[:seqlength],
        len_cwords = len_cwords,
        len_iwords = len_iwords
    )
    # print(*estimated_matrices, sep='\n')
    rankmax = np.linalg.matrix_rank(estimated_matrices[0][0])
    ranknum_fms, e_fms = numerical_rank_frob_mid_spec(estimated_matrices[0][0], seqlength, len_cwords, len_iwords, ret_bound=True)
    ranknum_bin, svals_bin, e_bin = numerical_rank_binomial(estimated_matrices[0][0], seqlength)
    print(f"{rankmax=}, {ranknum_bin=} at {e_bin=:.5f}, {ranknum_fms=} at {e_fms=:.5f}")#   {svals_bin[:5]=}")
    
    d1_range = range(2, min(rankmax, ranknum_bin + 1, 10))
    d1_range = range(max(ranknum_bin, 2), min(rankmax, max(ranknum_bin, 2) + 5))
    d1_range = range(max(ranknum_fms, 2), min(rankmax, max(ranknum_fms, 2) + 5))
    # d1_range = [2]
    for d1 in d1_range:
        # Estimate HMM
        learned_hmm = hmm.CategoricalHMM(n_components = d1, n_iter = 16)
        learned_hmm.fit(train_seq_modforhmm[:seqlength])
        hmm_nlls = []
        for test_seq_modforhmm in test_seqs_modforhmm:
            learned_hmm_nll_test = -learned_hmm.score(test_seq_modforhmm) / (test_seq_modforhmm.shape[0] * np.log(2))
            hmm_nlls.append(learned_hmm_nll_test)
        hmm_mean_nll = np.mean(hmm_nlls)
        hmm_std_nll = np.std(hmm_nlls)
        
        conv_hmm: DiscreteValuedOOM = dvoom_from_hmm(learned_hmm)
        fsp5_hmm = fix_pvec(quantify_distribution(5, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func))
        kldiv_hmm = kl_divergence(fsp5_source, fsp5_hmm)
        
        # if kldiv_hmm < best_hmm_kl: best_hmm, best_hmm_kl = conv_hmm, kldiv_hmm
        if (learned_hmm_nll_test-oom_model_mean_nll) < (best_hmm_nll-oom_model_mean_nll):
            best_hmm, best_hmm_kl, best_hmm_nll = conv_hmm, kldiv_hmm, learned_hmm_nll_test

        # Estimate OOM
        dvoom = DiscreteValuedOOM.from_data(
            obs = train_seq.sequence[:seqlength],
            target_dimension = d1,
            len_cwords = len_cwords,
            len_iwords = len_iwords,
            estimated_matrices = estimated_matrices
        )
        dvoom.normalize(ones_row=True)
        dvoom_nlls = [dvoom.compute(test_seq.sequence).nll_list[-1] for test_seq in test_seqs]
        dvoom_mean_nll = np.mean(dvoom_nlls)
        dvoom_std_nll = np.std(dvoom_nlls)

        fsp5_oom = fix_pvec(quantify_distribution(5, dvoom.start_state, dvoom.operators, dvoom.lin_func))
        kldiv_oom = kl_divergence(fsp5_source, fsp5_oom)
        # if kldiv_oom < best_oom_kl: best_oom, best_oom_kl = dvoom, kldiv_oom
        if abs(dvoom_mean_nll-oom_model_mean_nll) < (best_oom_nll-oom_model_mean_nll):
            best_oom, best_oom_kl, best_oom_nll = dvoom, kldiv_oom, dvoom_mean_nll
        # print(fsp5_oom)
        
        print(f"Metrics for length {seqlength:>10}: "
              f"HMM({d1=})  NLL = {hmm_mean_nll:.3f}+-{hmm_std_nll:.3f}   KL = {kldiv_hmm:.4f}      "
              f"OOM({d1=})  NLL = {dvoom_mean_nll:.3f}+-{dvoom_std_nll:.3f}  KL = {kldiv_oom:.4f}")

In [ ]:
print(best_hmm)

In [ ]:
print(best_oom)

In [ ]:
def mse(p, q):
    return np.mean((p - q) ** 2)

In [ ]:
q = 5

model_pvec = fix_pvec(quantify_distribution(q, oom_model.start_state, oom_model.operators, oom_model.lin_func))
besth_pvec = fix_pvec(quantify_distribution(q, best_hmm.start_state, best_hmm.operators, best_hmm.lin_func))
besto_pvec = fix_pvec(quantify_distribution(q, best_oom.start_state, best_oom.operators, best_oom.lin_func))

plt.plot(model_pvec, label="source (ground truth)")
plt.plot(besth_pvec, label="best hmm approximator")
plt.plot(besto_pvec, label="best oom approximator")
plt.title(f"Entries of the probability vectors for {q}-step predictive sequences")
plt.xticks(
    labels=[oom_model.observables[0].uid[1:] * q, oom_model.observables[-1].uid[1:] * q],
    ticks = [0, len(oom_model.observables)**q],
    rotation = 20
)

# plt.ylim([0, 1])
plt.xlim([0, len(oom_model.observables)**q])
# plt.axhline(fix_pvec(quantify_distribution(q, oom_model.start_state, oom_model.operators, oom_model.lin_func))[0], ls='--')
# plt.axhline(fix_pvec(quantify_distribution(q, oom_model.start_state, oom_model.operators, oom_model.lin_func))[1], ls='--')
# plt.plot(fix_pvec(quantify_distribution(q, best_oom.start_state, best_oom.operators, best_oom.lin_func))[::-1], label="best oom reversed")

plt.legend()

print(
    "KL = %.5f, MSE = %.10f for best HMM" % (kl_divergence(model_pvec, besth_pvec), mse(model_pvec, besth_pvec)),
    "KL = %.5f, MSE = %.10f for best OOM" % (kl_divergence(model_pvec, besto_pvec), mse(model_pvec, besto_pvec)),
    sep='\n'
)

plt.show()

# Fundamental Exploration

In [4]:
# Create stationary transition matrix (symmetric for easy stationary distribution)
transmat = np.array([[0.8, 0.2],  # From state 0: 80% stay, 20% to state 1
                     [0.2, 0.8]]) # From state 1: 20% to state 0, 80% stay

# Distinct emissions (easy to distinguish states)
emissionprob = np.array([[0.9, 0.1],  # State 0: 90% 'a', 10% 'b
                         [0.1, 0.9]]) # State 1: 10% 'a', 90% 'b'

# Stationary distribution (eigenvector for eigenvalue 1)
startprob = np.array([0.5, 0.5])  # Stationary for symmetric transitions

# Create and configure model
model = hmm.CategoricalHMM(n_components=2, implementation="log")
model.n_features = 2  # Number of observable symbols (a=0, b=1)
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

# Generate test sequence
observations, states = model.sample(1000)

# Verify parameters
print("True transition matrix:\n", transmat)
print("True emission matrix:\n", emissionprob)
print("First 10 observations:", observations[:10].flatten())
print("First 10 hidden states:", states[:10])

True transition matrix:
 [[0.8 0.2]
 [0.2 0.8]]
True emission matrix:
 [[0.9 0.1]
 [0.1 0.9]]
First 10 observations: [1 0 1 0 0 0 1 1 1 1]
First 10 hidden states: [1 1 1 0 0 0 1 1 1 1]


In [5]:
mfs = [sp.stats.uniform(loc=0, scale=1), sp.stats.uniform(loc=1, scale=1)]

In [6]:
conv_hmm_disc = dvoom_from_hmm(model)
conv_hmm = ContinuousValuedOOM.from_discrete_valued_oom(
    conv_hmm_disc,
    mfs
)

In [7]:
chgen = conv_hmm.generate(1000)

In [8]:
len_cwords = 2
len_iwords = 2
seqlen = 1000

print(chgen.sequence[:10], '\n', chgen.sequence_cont[:10], end='\n\n')

estimated_matrices = estimate_matrices_continuous(
    sequence   = chgen.sequence_cont[:seqlen],
    len_cwords = len_cwords,
    len_iwords = len_iwords,
    membership_functions = conv_hmm.membership_fns,
    observables = conv_hmm.observables
)
for item in estimated_matrices:
    if isinstance(item, dict):
        for key, val in item.items():
            print(key, val, sep='\n', end='\n\n')
    else: print(item, end='\n\n')

print("rank_bin:", *numerical_rank_binomial(estimated_matrices[0][0], seqlen))
print("rank_fms:", *numerical_rank_frob_mid_spec(estimated_matrices[0][0], seqlen, len_cwords, len_iwords, ret_bound=True))
print("rank_tru:", np.linalg.matrix_rank(estimated_matrices[0][0]))
print()

learned = ContinuousValuedOOM.from_data(
    chgen.sequence_cont[:seqlen],
    target_dimension = 2,
    len_cwords = len_cwords,
    len_iwords = len_iwords,
    observables = conv_hmm.observables,
    membership_functions = conv_hmm.membership_fns,
    estimated_matrices = estimated_matrices
)

q=1
test_seq = conv_hmm.generate(50000)
nll_conv_hmm = conv_hmm.compute(test_seq.sequence_cont).nll_list[-1]

print('\n\n' + ('*' * 50) + '\nBEFORE NORMALIZATION\n' + ('*' * 50) + '\n\n')

print(
    learned,
    conv_hmm,
    # fix_pvec(quantify_distribution(q, learned.start_state, learned.operators, learned.lin_func)),
    # fix_pvec(quantify_distribution(q, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func)),
    # kl_divergence(
    #     fix_pvec(quantify_distribution(q, learned.start_state, learned.operators, learned.lin_func)),
    #     fix_pvec(quantify_distribution(q, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func))
    # ),
    # fix_pvec(quantify_distribution(5, learned.start_state, learned.operators, learned.lin_func)),
    # fix_pvec(quantify_distribution(5, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func)),
    # kl_divergence(
    #     fix_pvec(quantify_distribution(5, learned.start_state, learned.operators, learned.lin_func)),
    #     fix_pvec(quantify_distribution(5, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func))
    # ),
    f"original model has NLL {nll_conv_hmm} and learned has NLL {learned.compute(test_seq.sequence_cont).nll_list[-1]}",
    sep='\n\n'
)

print('\n\n' + ('*' * 50) + '\nAFTER NORMALIZATION\n' + ('*' * 50) + '\n\n')

learned.normalize(ones_row=True)
print(
    learned,
    conv_hmm,
    # fix_pvec(quantify_distribution(q, learned.start_state, learned.operators, learned.lin_func)),
    # fix_pvec(quantify_distribution(q, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func)),
    # kl_divergence(
    #     fix_pvec(quantify_distribution(q, learned.start_state, learned.operators, learned.lin_func)),
    #     fix_pvec(quantify_distribution(q, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func))
    # ),
    # fix_pvec(quantify_distribution(5, learned.start_state, learned.operators, learned.lin_func)),
    # fix_pvec(quantify_distribution(5, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func)),
    # kl_divergence(
    #     fix_pvec(quantify_distribution(5, learned.start_state, learned.operators, learned.lin_func)),
    #     fix_pvec(quantify_distribution(5, conv_hmm.start_state, conv_hmm.operators, conv_hmm.lin_func))
    # ),
    f"original model has NLL {nll_conv_hmm} and learned has NLL {learned.compute(test_seq.sequence_cont).nll_list[-1]}",
    sep='\n\n'
)

[O1, O1, O1, O1, O2, O2, O2, O2, O2, O1] 
 [0.403267692336974, 0.10416824731331464, 0.33005924751460936, 0.921766128710205, 1.6549606110411323, 1.4661528878609498, 1.0003806974222136, 1.4075985991606257, 1.3366024033463306, 0.0204982689493034]



100%|█████████████████████████████████████████████████████████████████████████████| 996/996 [00:00<00:00, 15122.32it/s]

0 0 1.0
0 1 0.0
1 0 0.0
1 1 0.9999999999999084


0
[[0.17151454 0.03209629 0.06018054 0.05416249]
 [0.06118355 0.01705115 0.02607823 0.04513541]
 [0.03109328 0.02607823 0.01805416 0.07321966]
 [0.05616851 0.07321966 0.0441324  0.2106319 ]]

O1
[[0.12550201 0.02309237 0.04518072 0.03714859]
 [0.04618474 0.00903614 0.01506024 0.01706827]
 [0.02008032 0.00803213 0.01104418 0.01004016]
 [0.04116466 0.00903614 0.01506024 0.03514056]]

O2
[[0.01807229 0.01606426 0.01405622 0.03815261]
 [0.01305221 0.01004016 0.00401606 0.03514056]
 [0.01506024 0.02008032 0.01104418 0.05321285]
 [0.04016064 0.05321285 0.03313253 0.15763052]]

[[0.31931932 0.14914915 0.14814815 0.38338338]]

[[0.31931932]
 [0.14914915]
 [0.14814815]
 [0.38338338]]

rank_bin: 2 [294.28475368 138.94300945   2.86112885   0.83058658] 56.87764511077113
rank_fms: 2 0.024428881986806313
rank_tru: 4



**************************************************
BEFORE NORMALIZATION
**************************************************


<ContinuousValuedOOM object with dimension 2 and alphabet 

In [15]:
arr = np.ones(shape=(5,))
d = arr.size
arr[d]

IndexError: index 5 is out of bounds for axis 0 with size 5

In [21]:
svals_simd = np.linspace(1, 0, 10)
svals_simd

array([1.        , 0.88888889, 0.77777778, 0.66666667, 0.55555556, 0.44444444, 0.33333333, 0.22222222, 0.11111111, 0.        ])

In [25]:
rank_num = svals_simd.size
cutoff = 0
rank_num, cutoff

(10, 0)

In [ ]:

while rank_num > 0 and s2_tailsum <= cutoff:
    d_numrank -= 1
    s2_tailsum += s[d_numrank]**2

e = s[d_numrank] - (s2_tailsum**0.5 - e**0.5) / s[d_numrank] * (s[d_numrank] - (0 if d_numrank == len(s) else s[d_numrank+1]))
d_numrank += 1